In [1]:
from lightgbm import LGBMRegressor
import types
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd

In [2]:
#Cargo los sets
X_train = pd.read_csv('data/X_train_features.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_train = pd.read_csv('data/y_train.csv',header=None,squeeze=True,index_col=0)
X_test = pd.read_csv('data/X_test_features.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')
y_test = pd.read_csv('data/y_test.csv',header=None,squeeze=True,index_col=0)

In [3]:
X_submit = pd.read_csv('data/X_submit_features.csv', dtype={'id':'int',\
                                           'tipodepropiedad':'category',\
                                           'ciudad':'category', 'provincia':'category',\
                                           'antiguedad':'float', 'habitaciones':'float',\
                                           'garages':'float','banos':'float',\
                                           'metroscubiertos':'float', 'metrostotales':'float',\
                                           'idzona':'float',\
                                           'gimnasio':'bool', 'usosmultiples':'bool',\
                                           'piscina':'bool', 'escuelascercanas':'bool',\
                                           'centroscomercialescercanos':'bool'\
                                           }, index_col='id')

In [4]:
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [5]:
features_50 = pd.read_csv('data/features_importancia_tercera_etapa.csv',index_col=0).head(60)\
            .index.tolist()
features_50

['metroscubiertos',
 'banos',
 'metrostotales',
 'provincia__Distrito Federal',
 'idzona',
 'garages',
 'tipodepropiedad__Apartamento',
 'tipodepropiedad__Terreno',
 'habitaciones',
 'antiguedad',
 'año',
 'dia',
 'tipodepropiedad__Casa',
 'mes',
 'provincia__Edo. de México',
 'ciudad_3',
 'ciudad_2',
 'ciudad_8',
 'ciudad_4',
 'ciudad_9',
 'ciudad_7',
 'ciudad_10',
 'ciudad_5',
 'ciudad_6',
 'provincia__Nuevo León',
 'provincia__Jalisco',
 'tipodepropiedad__Terreno comercial',
 'centroscomercialescercanos',
 'provincia__Yucatán',
 'escuelascercanas',
 'piscina',
 'tipodepropiedad__Casa en condominio',
 'gimnasio',
 'tipodepropiedad__Edificio',
 'provincia__Querétaro',
 'tipodepropiedad__Local Comercial',
 'provincia__San luis Potosí',
 'provincia__Puebla',
 'provincia__Durango',
 'provincia__Coahuila',
 'tipodepropiedad__Oficina comercial',
 'tipodepropiedad__Bodega comercial',
 'provincia__Morelos',
 'usosmultiples',
 'provincia__Sonora',
 'provincia__Chihuahua',
 'provincia__Veracru

In [6]:
X_train_n = X_train[features_50]
X_test_n = X_test[features_50]

In [7]:
X_submit_n = X_submit[features_50]

In [8]:
feature_names = X_train_n.columns.tolist()
features_names_ascii = []

In [9]:
for name in feature_names:
    name = name.encode('ascii', 'ignore').decode('ascii')
    features_names_ascii.append(name)

In [10]:
X_train_n.columns = features_names_ascii

In [11]:
X_test_n.columns = features_names_ascii

In [12]:
X_submit_n.columns = features_names_ascii

In [13]:
def RMSE(actual, pred):
    return np.sqrt(mean_squared_error(actual, pred))

In [137]:
forest = LGBMRegressor(n_estimators=200, class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=25,
              min_child_samples=15, min_child_weight=0.001, min_split_gain=0.0,
              boosting_type='gbdt', n_jobs=5, num_leaves=1000, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
              subsample=1.0, subsample_freq=0)

In [138]:
%time forest.fit(X_train_n, y_train_log)

CPU times: user 52.4 s, sys: 268 ms, total: 52.7 s
Wall time: 11 s


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
              importance_type='split', learning_rate=0.05, max_depth=25,
              min_child_samples=15, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=200, n_jobs=5, num_leaves=1000, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=False,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [139]:
y_train_predict = forest.predict(X_train_n)
y_test_predict = forest.predict(X_test_n)

In [140]:
rmse_train=RMSE(y_train_log,y_train_predict)
print(rmse_train)

0.21989314207287547


In [141]:
rmse_test=RMSE(y_test_log,y_test_predict)
print(rmse_test)

0.31712820603110614


In [ ]:
0.21086491018527126

In [ ]:
0.31744670347298926

In [157]:
from sklearn.metrics import make_scorer
score_func = make_scorer(mean_squared_error, greater_is_better=False)

model = LGBMRegressor( 
    boosting_type="gbdt",
    objective='regression',
    is_unbalance=True, 
    random_state=10, 
    n_estimators=50,
    num_leaves=1000, 
    max_depth=25,
    feature_fraction=0.5,  
    bagging_fraction=0.8, 
    bagging_freq=15, 
    learning_rate=0.05,    
)

params_opt = {'n_estimators':range(50, 400, 80), 'num_leaves':range(20,1000,75)}
%time gridSearchCV = GridSearchCV(estimator = model, param_grid = params_opt, scoring=score_func)
%time gridSearchCV.fit(X_train_n,y_train_log)

CPU times: user 28 µs, sys: 0 ns, total: 28 µs
Wall time: 32.4 µs


/home/franco/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


CPU times: user 1h 52min 5s, sys: 32.8 s, total: 1h 52min 38s
Wall time: 23min 27s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LGBMRegressor(bagging_fraction=0.8, bagging_freq=15,
                                     boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0, feature_fraction=0.5,
                                     importance_type='split', is_unbalance=True,
                                     learning_rate=0.05, max_depth=25,
                                     min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=50,...
                                     num_leaves=1000, objective='regression',
                                     random_state=10, reg_alpha=0.0,
                                     reg_lambda=0.0, silent=True, subsample=1.0,
                                     subsample_for_bin=200000,
                                     subsample_freq=0),
            

In [158]:
gridSearchCV.cv_results_

{'mean_fit_time': array([ 0.95401446,  1.18331432,  1.40243856,  1.5041221 ,  1.6283462 ,
         1.71387021,  1.78017966,  2.04303678,  2.00360425,  2.09165994,
         2.17241748,  2.27412526,  2.37242953,  2.46032929,  1.39384015,
         2.11986335,  2.52969209,  2.89161595,  3.28137422,  3.65133063,
         3.8390797 ,  3.9941709 ,  4.4370234 ,  4.78256607,  5.15053113,
         5.19914977,  5.2842478 ,  6.04926626,  2.07155808,  2.95866895,
         3.50625666,  4.15265989,  4.89353156,  5.01089994,  5.50616558,
         5.8218116 ,  6.1799434 ,  6.98434313,  7.05445043,  8.35519218,
         8.22444169,  8.63321479,  2.55541039,  4.00606028,  4.77646796,
         5.38973721,  6.02487246,  6.80861076,  7.19317174,  7.80587403,
         8.42585874,  9.52539404,  9.66115491, 10.38559222, 11.62554884,
        12.05673321,  2.69075314,  4.81659754,  5.5951368 ,  6.3061072 ,
         7.36063576,  7.96745992,  9.18570916, 10.70430271, 10.90040127,
        11.3663679 , 11.8862385 , 

In [159]:
gridSearchCV.best_params_

{'n_estimators': 370, 'num_leaves': 845}

In [ ]:
#Correrlo con {'n_estimators': 370, 'num_leaves': 845} y los parametros anteriores

In [160]:
gridSearchCV.best_score_

-0.10037989135131214

In [161]:
RMSE(gridSearchCV.predict(X_test_n),y_test_log)

0.3117647022594891

In [162]:
RMSE(gridSearchCV.predict(X_train_n),y_train_log)

0.21033512994948192

In [ ]:
#Se puede ver un overfitteo en el set de train

In [ ]:
#Finalmente los parametros son

In [14]:
model = LGBMRegressor( 
    boosting_type="gbdt",
    objective='regression',
    is_unbalance=True, 
    random_state=10, 
    n_estimators=370,
    num_leaves=845, 
    max_depth=25,
    feature_fraction=0.5,  
    bagging_fraction=0.8, 
    bagging_freq=15, 
    learning_rate=0.05,    
)

In [15]:
%time model.fit(X_train_n, y_train_log)

CPU times: user 1min 48s, sys: 710 ms, total: 1min 49s
Wall time: 14.5 s


LGBMRegressor(bagging_fraction=0.8, bagging_freq=15, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.5,
              importance_type='split', is_unbalance=True, learning_rate=0.05,
              max_depth=25, min_child_samples=20, min_child_weight=0.001,
              min_split_gain=0.0, n_estimators=370, n_jobs=-1, num_leaves=845,
              objective='regression', random_state=10, reg_alpha=0.0,
              reg_lambda=0.0, silent=True, subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0)

In [16]:
y_train_predict = model.predict(X_train_n)
y_test_predict = model.predict(X_test_n)
y_submit_predict = model.predict(X_submit_n)

In [17]:
rmse_train=RMSE(y_train_log,y_train_predict)
print(rmse_train)

0.21073924704181274


In [18]:
rmse_test=RMSE(y_test_log,y_test_predict)
print(rmse_test)

0.3114459403799977


In [19]:
serie_submit = pd.Series(data=np.exp(y_submit_predict),index=X_submit.index,name='target')

In [21]:
df_entrega = serie_submit.reset_index()
df_entrega.head()

,id,target
0,4941,4.025212e+06
1,51775,8.652843e+05
2,115253,2.628328e+06
3,299321,1.144888e+06
4,173570,5.461169e+05


In [22]:
df_entrega.to_csv('data/submitLGBMfix.csv',index=False)